In [1]:
# Import libraries
import os
import json
import pandas as pd
from elasticsearch import Elasticsearch, ElasticsearchWarning
from tqdm.auto import tqdm
import minsearch
from sentence_transformers import SentenceTransformer

import warnings
warnings.filterwarnings("ignore", category=ElasticsearchWarning)

C:\Users\ADMIN\.virtualenvs\LLMs-Course-DataTalksClub-1-tkIJJ30C\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Read JSON document with generated ids

with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [3]:
# Define the model to use
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

In [4]:
v = model.encode('I just discovered the course. Can I still join?')
len(v) # dimensions

384

In [5]:
# Load the ground truth CSV file into a DataFrame 
df_ground_truth = pd.read_csv('ground-truth-data.csv')

ground_truth = df_ground_truth.to_dict(orient='records') # list of dictionaries

**Elastic Search**

In [6]:
# Initialize elastic search

es_client = Elasticsearch('http://localhost:9200') 

In [7]:
# Create an index mapping the question, text and question_text embeddings

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 384
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 384
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 384
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [8]:
# Index the documents

for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    # Create embeddings
    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qt)

100%|█████████████████████████████████████████████████████████████████████████| 948/948 [05:04<00:00,  3.12it/s]


In [9]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████████████████████████████████████| 948/948 [03:57<00:00,  4.00it/s]


In [10]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [11]:
def question_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_vector', v_q, course)

In [12]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [13]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [16]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [17]:
evaluate(ground_truth, question_vector_knn)

  0%|                                                                                  | 0/4627 [00:00<?, ?it/s]


BadRequestError: BadRequestError(400, 'parsing_exception', 'Unknown key for a START_OBJECT in [knn].')

In [ ]:
def text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)       

    return elastic_search_knn('text_vector', v_q, course)